# <div align="center" style="color: #ff5733;">New Pos Addition and Mapping</div>

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\SIL Sales\Shin Nebria\POS_Addition\Data\List of Store Opening (PowerBI) 2.xlsx", sheet_name = 'PowerBI')
df.head()

,POS NAME,STORE LOCATION,STATUS,DATE ACTIVATION,PURPLE KEY,REGION,TSM,Brand
0,Audionet Vistamall Bataan,Vista Mall Bataan,For Opening,2024-05-06,MJ8GPQU,North Luzon 2,Rachelle Belmonte,Rulls
1,Stormtech Vistamall Bataan,Vista Mall Cupang,For Opening,2024-05-06,MU1S8ZR,North Luzon 2,Rachelle Belmonte,Rulls
2,MBC Ballesteros Bazar,Standalone,For Opening,2024-05-06,M56AEK4,North Luzon 2,Rachelle Belmonte,Rulls
3,2nd MBC Ballesteros Cagayan,Standalone,For Opening,2024-05-06,MVZ2O4R,North Luzon 2,Rachelle Belmonte,Rulls
4,Oppo Kiosk Northstar Mall Ilagan,Xentro Mall Alibagu,For Opening,2024-05-06,M29DF4O,North Luzon 2,Rachelle Belmonte,Rulls


In [4]:
df.rename(columns = {'Brand': 'BrandStores'}, inplace = True)
df.columns

Index(['POS NAME', 'STORE LOCATION', 'STATUS', 'DATE ACTIVATION', 'PURPLE KEY',
       'REGION', 'TSM', 'BrandStores'],
      dtype='object')

In [5]:
df['BrandStores'].value_counts()

BrandStores
Rulls    30
Name: count, dtype: int64

In [6]:
df['PURPLE KEY'].value_counts()

PURPLE KEY
MJ8GPQU    1
MU1S8ZR    1
MISC3NG    1
M9QPNN0    1
MA7659P    1
M3IT8U0    1
M34LFKE    1
MS6UZIY    1
M890REN    1
M5L0OIX    1
MKVI0Y6    1
MUO270T    1
MF623DX    1
M5HZ3QW    1
MWX5VZO    1
MFAPDXS    1
MRWZN9D    1
MUWBVH7    1
ML0C3DB    1
MU93ADK    1
M964RJS    1
MJ2OCWF    1
MRK48XZ    1
M4KPCYL    1
MP30S8T    1
MO8LU72    1
M29DF4O    1
MVZ2O4R    1
M56AEK4    1
MOEPAHL    1
Name: count, dtype: int64

In [7]:
df = df[['BrandStores', 'POS NAME', 'STORE LOCATION', 'STATUS', 'DATE ACTIVATION', 'PURPLE KEY', 'REGION', 'TSM']]
df

,BrandStores,POS NAME,STORE LOCATION,STATUS,DATE ACTIVATION,PURPLE KEY,REGION,TSM
0,Rulls,Audionet Vistamall Bataan,Vista Mall Bataan,For Opening,2024-05-06,MJ8GPQU,North Luzon 2,Rachelle Belmonte
1,Rulls,Stormtech Vistamall Bataan,Vista Mall Cupang,For Opening,2024-05-06,MU1S8ZR,North Luzon 2,Rachelle Belmonte
2,Rulls,MBC Ballesteros Bazar,Standalone,For Opening,2024-05-06,M56AEK4,North Luzon 2,Rachelle Belmonte
3,Rulls,2nd MBC Ballesteros Cagayan,Standalone,For Opening,2024-05-06,MVZ2O4R,North Luzon 2,Rachelle Belmonte
4,Rulls,Oppo Kiosk Northstar Mall Ilagan,Xentro Mall Alibagu,For Opening,2024-05-06,M29DF4O,North Luzon 2,Rachelle Belmonte
5,Rulls,Rulls Robinson Place Santiago,Robinsons Santiago,For Opening,2024-05-06,MO8LU72,North Luzon 2,Rachelle Belmonte
6,Rulls,Audionet Robinson Place Santiago,Robinsons Santiago,For Opening,2024-05-06,MP30S8T,North Luzon 2,Rachelle Belmonte
7,Rulls,Techno Robinsons Place Santiago,Robinsons Santiago,For Opening,2024-05-06,M4KPCYL,North Luzon 2,Rachelle Belmonte
8,Rulls,Stormtech Rob Santiago,Robinsons Maharlika,For Opening,2024-05-06,MRK48XZ,North Luzon 2,Rachelle Belmonte
9,Rulls,Rulls Waltermart San Jose Nueva Ecija,Waltermart Maharlika,For Opening,2024-05-06,MJ2OCWF,North Luzon 2,Rachelle Belmonte


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   BrandStores      30 non-null     object        
 1   POS NAME         30 non-null     object        
 2   STORE LOCATION   30 non-null     object        
 3   STATUS           30 non-null     object        
 4   DATE ACTIVATION  30 non-null     datetime64[ns]
 5   PURPLE KEY       30 non-null     object        
 6   REGION           30 non-null     object        
 7   TSM              30 non-null     object        
dtypes: datetime64[ns](1), object(7)
memory usage: 2.0+ KB


In [9]:
df.rename(columns={'POS NAME':'POS', 'DATE ACTIVATION':'DateOpened', 'STORE LOCATION': 'StoreLocation', 'STATUS':'Status', 'PURPLE KEY':'PurpleKey', 'REGION':'Region'}, inplace = True)

In [10]:
import pandas as pd

df['DateOpened'] = pd.to_datetime(df['DateOpened'], format='%d-%b-%y')

In [11]:
print(df['DateOpened'].dtype)
# datetime64[ns]

datetime64[ns]


In [12]:
print(df['DateOpened'].iloc[0]) 
# 2024-01-15 00:00:00

2024-05-06 00:00:00


In [13]:
df['DateOpened'] = df['DateOpened'].dt.strftime('%Y-%m-%d')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   BrandStores    30 non-null     object
 1   POS            30 non-null     object
 2   StoreLocation  30 non-null     object
 3   Status         30 non-null     object
 4   DateOpened     30 non-null     object
 5   PurpleKey      30 non-null     object
 6   Region         30 non-null     object
 7   TSM            30 non-null     object
dtypes: object(8)
memory usage: 2.0+ KB


In [14]:
df['DateOpened'] = pd.to_datetime(df['DateOpened'], format='%Y-%m-%d')

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   BrandStores    30 non-null     object        
 1   POS            30 non-null     object        
 2   StoreLocation  30 non-null     object        
 3   Status         30 non-null     object        
 4   DateOpened     30 non-null     datetime64[ns]
 5   PurpleKey      30 non-null     object        
 6   Region         30 non-null     object        
 7   TSM            30 non-null     object        
dtypes: datetime64[ns](1), object(7)
memory usage: 2.0+ KB


In [16]:
df.head()

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,Rulls,Audionet Vistamall Bataan,Vista Mall Bataan,For Opening,2024-05-06,MJ8GPQU,North Luzon 2,Rachelle Belmonte
1,Rulls,Stormtech Vistamall Bataan,Vista Mall Cupang,For Opening,2024-05-06,MU1S8ZR,North Luzon 2,Rachelle Belmonte
2,Rulls,MBC Ballesteros Bazar,Standalone,For Opening,2024-05-06,M56AEK4,North Luzon 2,Rachelle Belmonte
3,Rulls,2nd MBC Ballesteros Cagayan,Standalone,For Opening,2024-05-06,MVZ2O4R,North Luzon 2,Rachelle Belmonte
4,Rulls,Oppo Kiosk Northstar Mall Ilagan,Xentro Mall Alibagu,For Opening,2024-05-06,M29DF4O,North Luzon 2,Rachelle Belmonte


In [17]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
client.query(sq)
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
try:
    d = client.query(sq).to_dataframe(progress_bar_type='tqdm')
    print("Table still exists")
except:
    print("Table dropped")

Query is running:   0%|          |Table dropped


In [18]:
sq = """create table prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1
as 
select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING;
"""
client.query(sq)



QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=66281f6e-7125-4aaa-adec-c82269ce39a6>

In [19]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
d = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of d is:\t{d.shape}")

Job ID 5ac5b40d-4556-4112-9528-bf4b33a3528d successfully executed: 100%|██████████|
Query is running:   0%|          |
Downloading: 100%|██████████|
The shape of d is:	(561, 8)


In [20]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1"""

dfsil = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID 62025b17-7298-4c3a-9fd8-12dd8bca824b successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [21]:
dfsil.BrandStores.value_counts()

BrandStores
Rulls               124
FC Home             109
Home Along           90
ESLA                 51
MemoXpress           37
Willy and Sons       29
Xtreme               22
Ansons               15
Homeworks            13
Automatic Centre     11
Kservico             10
Faith Glorious        9
Jhunpyo               8
Workstation           6
Power Mac Center      5
KUM Home Center       3
Sogo Home             3
Junliz                3
A2RJ                  3
Affordables           2
PHPS                  2
Home Factory          2
Hardware Sugar        2
Dizon Furniture       1
JTG                   1
Name: count, dtype: int64

In [22]:
dfsil.shape

(561, 8)

In [23]:
# Set table_id to the ID of the table to append to.
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1"

job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND"
)

job = client.load_table_from_dataframe(
    df, table_id, job_config=job_config
)  

job.result()  # Wait for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

Loaded 591 rows and 8 columns to prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1


In [24]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1"""
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')
print(f"The shape of the dataframe is: {df.shape}")

Job ID fd6c96f1-59b5-4e4e-b664-750911e7a31b successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is: (591, 8)


In [25]:
df.tail(10)

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
581,Rulls,Rulls Waltermart Paniqui,Waltermart Paniqui,For Opening,2024-05-06,MKVI0Y6,North Luzon 2,Rachelle Belmonte
582,Rulls,Mbc Waltermart Paniqui,Waltermart Paniqui,For Opening,2024-05-06,M5L0OIX,North Luzon 2,Rachelle Belmonte
583,Rulls,Samsung Waltermart Subic,Waltermart Mangan,For Opening,2024-05-06,M890REN,North Luzon 2,Rachelle Belmonte
584,Rulls,Rulls Kiosk Ayalamall Legazpi,Ayala Mall Albay,For Opening,2024-05-06,MS6UZIY,South Luzon 3,Jan jan Dela Cruz
585,Rulls,Mobilecraze Pacific Mall Legazpi,Pacific Mall Albay,For Opening,2024-05-06,M34LFKE,South Luzon 3,Jan jan Dela Cruz
586,Rulls,Oppo Kiosk Ayala Legazpi,Standalone,For Opening,2024-05-06,M3IT8U0,South Luzon 3,Jan jan Dela Cruz
587,Rulls,Oppo Kiosk Pacific Mall Legazpi,Pacific Mall Legazpi,For Opening,2024-05-06,MA7659P,South Luzon 3,Jan jan Dela Cruz
588,Rulls,Tecno Kiosk Rob Dasma,Robinsons Dasmarinas,For Opening,2024-05-06,M9QPNN0,South Luzon 1 (A),Aubrey Lalong Isip
589,Rulls,Rulls Centrall Mall,Central Mall Dasma,For Opening,2024-05-06,MISC3NG,South Luzon 1 (A),Aubrey Lalong Isip
590,Rulls,Stormtech Rob Dasma,Robinsons Dasmarinas,For Opening,2024-05-06,MOEPAHL,South Luzon 1 (A),Aubrey Lalong Isip


In [26]:
df['BrandStores'].value_counts()

BrandStores
Rulls               154
FC Home             109
Home Along           90
ESLA                 51
MemoXpress           37
Willy and Sons       29
Xtreme               22
Ansons               15
Homeworks            13
Automatic Centre     11
Kservico             10
Faith Glorious        9
Jhunpyo               8
Workstation           6
Power Mac Center      5
KUM Home Center       3
Sogo Home             3
Junliz                3
A2RJ                  3
Affordables           2
PHPS                  2
Home Factory          2
Hardware Sugar        2
Dizon Furniture       1
JTG                   1
Name: count, dtype: int64

In [27]:
# Find duplicate values in 'POS' column
duplicates = df[df.duplicated('POS', keep=False)]

# Print the entire rows for duplicate entries in 'POS'
if not duplicates.empty:
    print("Duplicate values in 'POS' column:")
    duplicates.to_csv("Temp.csv")
else:
    print("No duplicate values found in 'POS' column.")
    
dd = pd.read_csv("Temp.csv")
dd.sort_values(by='POS')

Duplicate values in 'POS' column:


,Unnamed: 0,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,2,Hardware Sugar,Hardware Sugar Makati,NaN,NaN,2022-07-04,MV4UO0X,NaN,NaN
1,3,Hardware Sugar,Hardware Sugar Makati,NaN,NaN,2022-07-04,M9ZK6IU,NaN,NaN
12,521,Home Along,Home Along - Mayon,Standalone,POS Operational,2022-07-01,MGP2AQI,NCR North,Ben Antonio Lacsamana
13,522,Home Along,Home Along - Mayon,Standalone,POS Operational,2022-07-01,MN3GL66,NCR North 2,Ben Antonio Lacsamana
14,530,Home Along,Home Along Malanday,Standalone,POS Operational,2022-07-11,MNBW4RZ,NCR North,Ben Antonio Lacsamana
15,531,Home Along,Home Along Malanday,NaN,NaN,2022-07-11,MURFAIV,NCR North,Ben Antonio Lacsamana
4,446,Home Along,Home Along Muntinlupa,Standalone,POS Operational,2022-03-26,MU82X9Z,NCR South,Klein Justine Tamayo
5,448,Home Along,Home Along Muntinlupa,Standalone,POS Operational,2022-03-26,MDGYHQJ,NCR South,Klein Justine Tamayo
9,460,Home Along,Home Along Puregold Taguig,Standalone,POS Operational,2022-07-01,MRGP0WQ,NCR South,Klein Justine Tamayo
10,461,Home Along,Home Along Puregold Taguig,Standalone,POS Operational,2022-07-01,MDO3KI4,NCR South,Klein Justine Tamayo


In [28]:
pd.set_option('display.max_rows', None)
# Count the occurrences of each value in the 'PurpleKey' column
purple_key_counts = df['PurpleKey'].value_counts()

# Filter to include only values with count >= 2
filtered_counts = purple_key_counts[purple_key_counts >= 2]

# Display the filtered counts
print(filtered_counts)


Series([], Name: count, dtype: int64)


In [29]:
df[df['PurpleKey']== 'MKVI0Y6']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
581,Rulls,Rulls Waltermart Paniqui,Waltermart Paniqui,For Opening,2024-05-06,MKVI0Y6,North Luzon 2,Rachelle Belmonte


In [30]:
sq = """drop table prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING;"""

client.query(sq)


QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=024a0b43-0157-472b-a37a-9a15c13335b1>

In [31]:
sq = """create table prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING as select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
client.query(sq)


QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=f6998863-c1ca-46ba-b380-5cbbdb9b9c54>

In [32]:
client.query("commit")

QueryJob<project=prj-prod-dataplatform, location=US, id=7d6eae06-416c-4cc1-b88d-2ec7c08e1034>

In [35]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING;"""

df = client.query(sq).to_dataframe(progress_bar_type='tqdm')
df.shape

Job ID fc256123-5197-4d50-b09a-9802f0873fed successfully executed: 100%|██████████|
Downloading: 100%|██████████|


(591, 8)

In [38]:
df[df['BrandStores']=='Rulls'].sort_values(by='DateOpened')

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
7,Rulls,Rulls Sta Lucia,Sta Lucia Mall,For Opening,2024-04-15,M58WV1L,NCR East,Albee David
385,Rulls,Rulls Waltermart Dasma,Waltermart Dasma,For Opening,2024-04-15,MPKQCNB,South Luzon 1 (A),Aubrey Lalong Isip
324,Rulls,Vivo Apalit,Savemore Apalit,For Opening,2024-04-15,MQULK65,North Luzon 2,Rachelle Belmonte
322,Rulls,Audionet Marquee Mall,Marque Mall,For Opening,2024-04-15,MCAZ46R,North Luzon 2,Rachelle Belmonte
321,Rulls,Audionet Apalit,Savemore Apalit,For Opening,2024-04-15,M9T6HF8,North Luzon 2,Rachelle Belmonte
320,Rulls,MBC Apalit,Savemore Apalit,For Opening,2024-04-15,MYIEOPS,North Luzon 2,Rachelle Belmonte
319,Rulls,Oppo Marquee Mall,Marque Mall,For Opening,2024-04-15,MYIF18A,North Luzon 2,Rachelle Belmonte
318,Rulls,Vivo Exhibit Marquee Mall,Marque Mall,For Opening,2024-04-15,MPORV4O,North Luzon 2,Rachelle Belmonte
317,Rulls,Rulls Apalit,Savemore Apalit,For Opening,2024-04-15,MA0C1DZ,North Luzon 2,Rachelle Belmonte
312,Rulls,Mobilecraze Marquee Mall,Marque Mall,For Opening,2024-04-15,MP1VR8S,North Luzon 2,Rachelle Belmonte


In [39]:
df.head(10)

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,FC Home,FC Home Center Balibago,LOT 5 BLK 9 GARNET ST. RHINELAND COMMERCIAL CO...,Operational,2023-08-01,MOG243Y,South Luzon 2,Klide Alberto
1,Xtreme,Xtreme Appliances Anabu,None,None,2022-08-27,MR2YDE8,None,None
2,Hardware Sugar,Hardware Sugar Makati,None,None,2022-07-04,MV4UO0X,None,None
3,Hardware Sugar,Hardware Sugar Makati,None,None,2022-07-04,M9ZK6IU,None,None
4,Home Along,Home Along Sumulong,None,None,2022-07-12,M2G78K4,None,None
5,Dizon Furniture,Dizon Furniture Tiendesitas,None,None,2022-07-23,MOE3JQH,None,None
6,Xtreme,Xtreme Appliances Lucky Gold Plaza,Lucky Gold Plaza,For Opening,2024-04-14,MO3AHGW,NCR East,Albee David
7,Rulls,Rulls Sta Lucia,Sta Lucia Mall,For Opening,2024-04-15,M58WV1L,NCR East,Albee David
8,MemoXpress,Memoxpress Sta Lucia East Grand Mall,Sta Lucia Mall,For Opening,2024-04-15,MRTAIQG,NCR East,Albee David
9,Ansons,Ansons Capitol Commons,Capitol Commons,For Opening,2024-04-15,M9EX1KL,NCR East,Albee David


In [40]:
df.sort_values('DateOpened', ascending=False)

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
356,Rulls,MBC Ballesteros Bazar,Standalone,For Opening,2024-05-06,M56AEK4,North Luzon 2,Rachelle Belmonte
342,Rulls,Rulls Waltermart San Jose Nueva Ecija,Waltermart Maharlika,For Opening,2024-05-06,MJ2OCWF,North Luzon 2,Rachelle Belmonte
336,Rulls,Rulls Robinson Gapan,Robinsons Bayanihan,For Opening,2024-05-06,MU93ADK,North Luzon 2,Rachelle Belmonte
337,Rulls,Rulls Waltermart Capas,Marque Mall,For Opening,2024-05-06,MUO270T,North Luzon 2,Rachelle Belmonte
338,Rulls,Techno Robinsons Place Santiago,Robinsons Santiago,For Opening,2024-05-06,M4KPCYL,North Luzon 2,Rachelle Belmonte
339,Rulls,Oppo Kiosk Northstar Mall Ilagan,Xentro Mall Alibagu,For Opening,2024-05-06,M29DF4O,North Luzon 2,Rachelle Belmonte
340,Rulls,Audionet Harbor Point,Harbor Point Mall Subic,For Opening,2024-05-06,MUWBVH7,North Luzon 2,Rachelle Belmonte
341,Rulls,2nd MBC Ballesteros Cagayan,Standalone,For Opening,2024-05-06,MVZ2O4R,North Luzon 2,Rachelle Belmonte
343,Rulls,Stormtech Rob Santiago,Robinsons Maharlika,For Opening,2024-05-06,MRK48XZ,North Luzon 2,Rachelle Belmonte
346,Rulls,Digibabe Robinsons Pangasinan,Robinsons Pangasinan,For Opening,2024-05-06,MF623DX,North Luzon 2,Rachelle Belmonte


In [41]:
df[df['PurpleKey'] =='M3Z10RP']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
466,Ansons,Ansons Greenhills,Greenhills,For Opening,2024-04-15,M3Z10RP,NCR South 1,Klein Justine Tamayo


In [42]:
df[df['BrandStores']=='ESLA']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
30,ESLA,ESLA Malabon,Standalone,For Opening,2024-04-22,M56ROOZ,NCR East,Albee David
31,ESLA,ESLA Balanga 2,Standalone,For Opening,2024-04-22,MLANGU7,NCR East,Albee David
47,ESLA,ESLA Montalban 2,Standalone,For Opening,2024-02-15,M9Q3NCO,NCR East,Charm Moltio
48,ESLA,ESLA Antipolo,Standalone,For Opening,2024-02-15,MDT8FNQ,NCR East,Charm Moltio
49,ESLA,ESLA San Mateo 1,Standalone,For Opening,2024-02-15,MD6S28C,NCR East,Charm Moltio
50,ESLA,ESLA Marikina 2,Standalone,For Opening,2024-02-15,MTGEZ1S,NCR East,Charm Moltio
51,ESLA,ESLA Taytay,Standalone,For Opening,2024-02-15,MWI9HXN,NCR East,Charm Moltio
52,ESLA,ESLA Pasig 2,Standalone,For Opening,2024-02-15,MRX4NDS,NCR East,Charm Moltio
53,ESLA,ESLA Montalban 1,Standalone,For Opening,2024-02-15,MO9V7YJ,NCR East,Charm Moltio
54,ESLA,ESLA Cogeo,Standalone,For Opening,2024-02-15,M9N5R2P,NCR East,Charm Moltio
